# Финальный проект: SQL.

**Постановка задачи:**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных:**

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## 1 шаг. Подключаемся к базе данных:

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas

Выедем первые 5 строк из таблицы books:

In [2]:
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Выедем первые 5 строк из таблицы authors:

In [3]:
query = '''SELECT * FROM authors LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Выедем первые 5 строк из таблицы publishers:

In [4]:
query = '''SELECT * FROM publishers LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Выедем первые 5 строк из таблицы ratings:

In [5]:
query = '''SELECT * FROM ratings LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Выедем первые 5 строк из таблицы reviews:

In [6]:
query = '''SELECT * FROM reviews LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Задача № 1:** Посчитаем, сколько книг вышло после 1 января 2000 года:

In [8]:
query = '''
SELECT  COUNT(book_id) AS books_count
FROM books 
WHERE CAST(publication_date AS DATE) > CAST('2000-01-01' AS DATE)
'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,books_count
0,819


**Вывод:** После 1 января 2000 года вышло 819 книг.

**Задача № 2:** Для каждой книги посчитаем количество обзоров и среднюю оценку:

In [9]:
query = '''
SELECT  b.book_id,
        b.title,
        AVG(rating) AS avg_rat,
        COUNT(DISTINCT(review_id)) AS rew_cont
FROM books AS b
LEFT JOIN ratings AS rt ON rt.book_id = b.book_id
LEFT JOIN reviews AS rw ON b.book_id = rw.book_id
GROUP BY b.book_id
ORDER BY rew_cont DESC, avg_rat DESC

'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,avg_rat,rew_cont
0,948,Twilight (Twilight #1),3.662500,7
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
2,299,Harry Potter and the Chamber of Secrets (Harry...,4.287500,6
3,656,The Book Thief,4.264151,6
4,734,The Glass Castle,4.206897,6
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,4.000000,0
996,387,Leonardo's Notebooks,4.000000,0
997,221,Essential Tales and Poems,4.000000,0
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,3.666667,0


**Вывод:** Больше всего обзоров написано на книгу "Twilight" ("Сумерки") - 7 штук.

**Задача № 3:** Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры:

In [10]:
query = '''
SELECT p.publisher_id,
       p.publisher,
       COUNT(distinct(b.book_id))
FROM books AS b
LEFT JOIN publishers AS p ON p.publisher_id = b.publisher_id
WHERE num_pages > 50
GROUP BY p.publisher_id
ORDER BY COUNT(distinct(b.book_id)) DESC
LIMIT 1
'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,count
0,212,Penguin Books,42


**Вывод:** Наибольшее число книг, толще 50 страниц, выпустило издательство "Penguin Books" - 42 шт.

**Задача № 4:** Определим автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками:

In [11]:
query = '''
SELECT  a.author,
        AVG(r.rating),
        COUNT(r.rating_id)
FROM books AS b
LEFT JOIN authors AS a ON a.author_id = b.author_id
LEFT JOIN ratings AS r ON r.book_id = b.book_id
WHERE b.book_id IN (
    SELECT book_id
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(book_id) > 50 
    )
GROUP BY a.author
ORDER BY AVG(r.rating) DESC
LIMIT 1
'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg,count
0,J.K. Rowling/Mary GrandPré,4.287097,310


**Вывод:** Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré (среди книг имеющих более 50 оценок)

**Задача № 5:** Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок:

In [12]:
query = '''
WITH 
rw AS (SELECT  username,
        COUNT(review_id) AS cont
    FROM reviews
    GROUP BY username
    ORDER BY cont DESC),

rt AS (SELECT  username
    FROM ratings 
    GROUP BY username
    HAVING COUNT(rating_id) > 48)

SELECT AVG(rw.cont) AS avg_count
FROM rt LEFT OUTER JOIN rw ON rt.username=rw.username

'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,avg_count
0,24.0


**Вывод:**  Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24.

**ОБЩИЙ ВЫВОД:** 

Нами была проанализирована база данных с информацией о книгах, издательствах, авторах, а также пользовательских обзоры книг:
- Выведены для ознакомления первые 5 строк каждой таблицы.
- Посчитано сколько книг выпущено после 1 января 2000 года - 819 книг.
- Найдена книга с самым большим количеством обзоров  - "Twilight" ("Сумерки") - 7 штук.
- Определино издательство, которое выпустило наибольшее число книг за исключением брошюр - "Penguin Books" - 42 шт.
- Определен автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré (среди книг имеющих более 50 оценок).
- Посчитано среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24 обзора.